## Init and import

In [2]:
# @title Imports
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)
import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

In [3]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

/Users/socialai/.pyenv/versions/3.11.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:13b',
    model_name='mixtral'
    # streaming=True
)


# import dotenv
# import os
# dotenv.load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# from concordia.language_model import gpt_model
# model = gpt_model.GptLanguageModel(
#     api_key=api_key,
#     model_name='gpt-4',
# )

In [5]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])


In [6]:
measurements = measurements_lib.Measurements()

In [40]:
# import importlib
import custom_components.quiz_metric as qm
# from concordia.language_model import ollama_model
# importlib.reload(ollama_model)
importlib.reload(qm)


model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:13b',
    model_name='mixtral'
    # streaming=True
)
# Answering big 5 test quesitons, 10questions
# Ocean score
# Extreme personalities 
context = """
Alice is extroverted, agreeable, conscientious, emotionally stable, and open to experiences. 

Here are a number of personality traits that may or may not apply to Alice. Please write a number next to each statement to indicate the extent to which Alice agree or disagree with that statement. Alice should rate the extent to which the pair of traits applies to hdf, even if one characteristic applies more strongly than the other.

1 = Disagree strongly
2 = Disagree moderately
3 = Disagree a little
4 = Neither agree nor disagree
5 = Agree a little
6 = Agree moderately
7 = Agree strongly
"""

quiz_metric = qm.QuizMetric(model, "Alice", clock, "./custom_components/big_five_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)


Answer: e
Sample:  e) 6

Based on the description of Alice being extroverted, I would rate her as agreeing moderately or a 6 with being extraverted and enthusiastic.
Question: From 1 to 7, rate how much you see yourself as 'Extraverted, enthusiastic'
Alice's answer: 6
Correct answer: 1

5
Answer: d
Sample:  Based on the description of Alice's personality, it is unlikely that she would identify as 'Critical, quarrelsome'. The traits given for Alice suggest that she is agreeable and emotionally stable, which are qualities that are generally not associated with being critical or quarrelsome. Therefore, I would choose option (d) 1, which represents "Disagree strongly".
Question: From 1 to 7, rate how much you see yourself as 'Critical, quarrelsome'
Alice's answer: 1
Correct answer: 1

0
Answer: f
Sample:  Based on the description of Alice as extroverted, agreeable, conscientious, emotionally stable, and open to experiences, I would rate her as a "6" for being 'Dependable, self-disciplined'

In [42]:
import pprint
import importlib
import custom_components.quiz_metric as qm
importlib.reload(qm)

results = quiz_metric.get_results()
personality_scores = quiz_metric.get_personality_scores()
# Results is an array of dicts
# agent_id, agent_name, question, answer, true_extraversion, measured_extra
import csv
with open('./results/big_five_personality.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Conscientiousness', 'Agreeableness', 'Emotional Stability', 'Openness to Experience', 'Extraversion'])
    writer.writerow(personality_scores)
    writer.writerow(['agent_name', 'agent_id', 'question', 'answer', 'correct_answer'])
    writer.writerows(results)